<a href="https://colab.research.google.com/github/jiayuanteng/Hands_on_ML2/blob/main/5_Support_Vector_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iris dataset


In [14]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC



# Support Vector Machine (Soft Margin Classifier)
One can work out oportimization problem of SVM as the Lagrange function below, where n is # of instances in the training data and p is # of features. The first term is a loss function (hinge loss). The second term is a regularization.  
  

Ł = $\sum_{n=1}^{n}max[0, 1-y_{i}(\beta_{0}+
\beta_{1}x_{i1} +...+ \beta_{p}x_{ip})]
+ λ\sum_{j=1}^{p}\beta_{j}^2$


λ is nonnegative tuning parameter. It is equivalent to the cost parameter C that regularizes # of violations to the margin. When λ is large, β is small and more voilations to margin are tolerated and a low-variance and high-margin classifier will result.





In [4]:
iris = datasets.load_iris()
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [5]:
iris['DESCR']

'.. _iris_dataset:\n\nIris plants dataset\n--------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n                \n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)\n    ============== ==== =

In [9]:
iris['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [10]:
# select petal length, petal width in X
# select iris virginica in y

X = iris['data'][:, (2, 3)]
y = (iris['target'] == 2).astype(np.float64)

In [16]:
svm_clf = Pipeline([
                    ("scaler", StandardScaler()),
                    ("linear_svc", SVC(C=1, kernel = 'linear')),

])
svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', SVC(C=1, kernel='linear'))])

In [13]:
svm_clf.predict([[5.5, 1.7]])

array([1.])